In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# ----------------------------
# CONFIGURACIÓN
# ----------------------------
IM_SIZE = 64

# ----------------------------
# CARGAR MODELO Y PCA
# ----------------------------
model = load_model("sign_model.h5")
pca_components = np.load("pca_components.npy")
pca_mean = np.load("pca_mean.npy")
classes = np.load("label_classes.npy", allow_pickle=True)

# ----------------------------
# APLICAR PCA
# ----------------------------
def apply_pca(flat_image):
    return (flat_image - pca_mean) @ pca_components.T

# ----------------------------
# CARGAR Y PROCESAR IMAGEN
# ----------------------------
def preprocess_image(path):
    img = cv2.imread(path)

    if img is None:
        raise ValueError(f"No se pudo abrir la imagen: {path}")

    img = cv2.resize(img, (IM_SIZE, IM_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype("float32") / 255.0

    flat = img.flatten()

    # reducir usando PCA
    pca_vec = apply_pca(flat)

    return pca_vec.reshape(1, -1)

# ----------------------------
# PREDICCIÓN
# ----------------------------
def predict(path):
    vec = preprocess_image(path)
    pred = model.predict(vec)
    idx = np.argmax(pred)
    letter = classes[idx]
    confidence = pred[0][idx]

    print("----------------------------")
    print(f"Imagen: {path}")
    print(f"Letra predicha: {letter}")
    print(f"Confianza: {confidence:.4f}")
    print("----------------------------")

    return letter


# ----------------------------
# EJEMPLO DE USO
# ----------------------------
if __name__ == "__main__":
    test_image = "Image/Z/15.png"  
     # cambia esto por la imagen a probar
    predict(test_image)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
----------------------------
Imagen: Image/Z/15.png
Letra predicha: Z
Confianza: 0.9968
----------------------------


## Random Forest

In [2]:
import numpy as np
import joblib

# Cargar modelos
rf = joblib.load("rf_model.pkl")
scaler = joblib.load("rf_scaler.pkl")

pca_components = np.load("rf_pca_components.npy")
pca_mean = np.load("rf_pca_mean.npy")
classes = np.load("rf_label_classes.npy", allow_pickle=True)

# PCA manual
def apply_pca(vec):
    return (vec - pca_mean) @ pca_components.T


def predict(values_dict):
    # Convertir dict → vector
    keys = [
        "x","y","z",
        "thumb","fore","index","ring","little",
        "thumb2","fore2","index2","ring2","little2",
        "keycode","gs1","gs2","sign",
        "giroX","giroY","giroZ"
    ]

    vec = np.array([values_dict[k] for k in keys], dtype="float32")

    # Normalizar
    vec = scaler.transform([vec])[0]

    # PCA
    vec_pca = apply_pca(vec)

    # Predicción
    idx = rf.predict([vec_pca])[0]
    return classes[idx]


In [3]:
pred = predict({
    "x": -120, "y": 80, "z": 60,
    "thumb": 0,"fore":0,"index":0,"ring":0,"little":0,
    "thumb2":20,"fore2":100,"index2":100,"ring2":100,"little2":100,
    "keycode":0,"gs1":0,"gs2":1,"sign":0,
    "giroX":0,"giroY":0,"giroZ":0
})

print("Letra predicha:", pred)


Letra predicha: A


## SVM

df = pd.read_csv("dataset.csv")

fila = df.iloc[100]  # ejemplo

pred = svm_predict_from_row(fila)
print("Predicción:", pred)


In [4]:
import joblib
import numpy as np
import cv2



def svm_predict_from_image(img_path):
    import cv2
    import numpy as np
    import joblib

    # Cargar modelos
    svm = joblib.load("svm_model.pkl")
    scaler = joblib.load("svm_scaler.pkl")
    pca = joblib.load("svm_pca.pkl")
    label_classes = np.load("svm_label_classes.npy", allow_pickle=True)

    # Procesar imagen
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img = img.flatten() / 255.0

    # Número de features que se usaron en el entrenamiento
    n_features_model = scaler.n_features_in_

    # Calculamos cuántas features numéricas necesita el dummy
    n_csv_features = n_features_model - len(img)

    csv_dummy = np.zeros((n_csv_features,), dtype=float)

    X = np.hstack([img, csv_dummy]).reshape(1, -1)

    # Pasar por scaler y PCA
    X_scaled = scaler.transform(X)
    X_pca = pca.transform(X_scaled)

    # Predicción
    pred_idx = svm.predict(X_pca)[0]
    pred_label = label_classes[pred_idx]

    print("Predicción:", pred_label)
    return pred_label




In [5]:
svm_predict_from_image("Image/I/10.png")

Predicción: I


'I'